In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from tqdm import tqdm
from scipy import stats
pd.set_option('display.max_columns', 100)

In [2]:
df = dd.read_csv('data/*matches*.csv', dtype={'home_team_goal_timings': 'object', 
                                             'away_team_goal_timings': 'object'})

In [3]:
df_players = dd.read_csv('data/*players*.csv', dtype={'season': 'object'})

In [4]:
df = df.compute()

In [5]:
df_players = df_players.compute()
df_players.dropna(subset=['full_name'], inplace=True)

In [6]:
df_matches_euro = pd.read_csv('euro-data/fixtures-1.csv')
df_teams_euro = pd.read_csv('euro-data/teams-1.csv')
df_players_euro = pd.read_csv('euro-data/players-active-1.csv')

In [7]:
df_matches_euro.head()

,mId,gmIsCurrent,gmIsLocked,gdId,isFeedLive,isLive,dateTime,dateTimeLock,htId,htName,htShortName,htCCode,atId,atName,atShortName,atCCode,gameNo,mdName,htScore,atScore,htAggScore,atAggScore,aggDescription,aggFlag,matchStatus,teamSc,teamScStartDate,teamScEndDate,lineupAnnounced,isMatchPostponed,groupId,groupName,stadiumId,stadiumName,stadiumThumb,venueId,venueName,venueCountryCode,mdId
0,2024447,1,0,1,0,0,06/11/2021 21:00:00,06/11/2021 21:00:00,135,Turkey,Turkey,TUR,66,Italy,Italy,ITA,33673,Match 1,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,2006438,Group A,57775,Olimpico in Rome,https://img.uefa.com/imgml/stadium/matchinfo/w...,2637,Rome,ITA,1
1,2024448,0,0,2,0,0,06/12/2021 15:00:00,06/12/2021 15:00:00,144,Wales,Wales,WAL,128,Switzerland,Switzerland,SUI,33673,Match 4,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,2006438,Group A,250002745,Bakı Olimpiya Stadionu,https://img.uefa.com/imgml/stadium/matchinfo/w...,1162,Baku,AZE,1
2,2024449,0,0,2,0,0,06/12/2021 18:00:00,06/12/2021 18:00:00,35,Denmark,Denmark,DEN,42,Finland,Finland,FIN,33673,Match 3,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,2006439,Group B,63462,Parken,https://img.uefa.com/imgml/stadium/matchinfo/w...,1449,Copenhagen,DEN,1
3,2024450,0,0,2,0,0,06/12/2021 21:00:00,06/12/2021 21:00:00,13,Belgium,Belgium,BEL,57451,Russia,Russia,RUS,33673,Match 2,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,2006439,Group B,250003363,Gazprom Arena,https://img.uefa.com/imgml/stadium/matchinfo/w...,2850,St Petersburg,RUS,1
4,2024451,0,0,3,0,0,06/13/2021 15:00:00,06/13/2021 15:00:00,39,England,England,ENG,56370,Croatia,Croatia,CRO,33673,Match 7,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,2006441,Group D,1100043,Wembley Stadium,https://img.uefa.com/imgml/stadium/matchinfo/w...,2116,London,ENG,1


In [8]:
df_teams_euro.head()

,id,webName,offName,shortName,typeIsNational,countryCode,isEliminated
0,8,Austria,Austria,AUT,1,AUT,NaN
1,13,Belgium,Belgium,BEL,1,BEL,NaN
2,35,Denmark,Denmark,DEN,1,DEN,NaN
3,39,England,England,ENG,1,ENG,NaN
4,42,Finland,Finland,FIN,1,FIN,NaN


In [9]:
df_players_euro.head()

,id,pDName,pFName,latinName,tName,tId,teamPlayed,cCode,skill,value,isActive,selPer,mdId,totPts,gS,assist,cS,gC,yC,rC,oG,pS,pC,pE,saves,pM,bR,gOB,mOM,mOMPts,pStatus,matchAtd,trained,isPlayed,selInPer,selOutPer,upcomingMatchesList,currentMatchesList,avgPlayerPts,avgPlayerValue,lastGdPoints,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15
0,63706,C. Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo,Portugal,110,0,POR,4,12.0,1,26.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '1', 'tId': '110', 'tSCode': 'Portug...","[{'mdId': '1', 'tId': '110', 'tSCode': 'Portug...",0.0,0.0,0.0,1.0,0.0,15.0,1.3,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,250076574,K. Mbappé,Kylian Mbappé,Kylian Mbappe,France,43,0,FRA,4,12.0,1,34.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '1', 'tId': '43', 'tSCode': 'France'...","[{'mdId': '1', 'tId': '43', 'tSCode': 'France'...",0.0,0.0,0.0,1.0,0.0,10.0,0.9,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,250016833,H. Kane,Harry Kane,Harry Kane,England,39,0,ENG,4,11.5,1,43.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '1', 'tId': '39', 'tSCode': 'England...","[{'mdId': '1', 'tId': '39', 'tSCode': 'England...",0.0,0.0,0.0,1.0,0.0,17.0,1.7,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,250002096,R. Lewandowski,Robert Lewandowski,Robert Lewandowski,Poland,109,0,POL,4,11.5,1,16.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '1', 'tId': '109', 'tSCode': 'Poland...","[{'mdId': '1', 'tId': '109', 'tSCode': 'Poland...",0.0,0.0,0.0,1.0,0.0,6.0,0.8,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,250010802,R. Lukaku,Romelu Lukaku,Romelu Lukaku,Belgium,13,0,BEL,4,11.0,1,42.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '1', 'tId': '13', 'tSCode': 'Belgium...","[{'mdId': '1', 'tId': '13', 'tSCode': 'Belgium...",0.0,0.0,0.0,1.0,0.0,16.0,2.2,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
df.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1470582000,Aug 07 2016 - 3:00pm,complete,NaN,Leicester City,Manchester United,Craig Pawson,NaN,0.0,0.0,0.00,3.00,1,2,3,1,0,1,52,"32,83",7,2,3,0,1,0,1,2,0,1,9,7,4,5,5,2,10,9,46,54,0.00,0.00,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,Wembley Stadium (London)
0,1502024400,Aug 06 2017 - 1:00pm,complete,NaN,Arsenal,Chelsea,Robert Madley,NaN,0.0,0.0,1.00,1.00,1,1,2,0,0,0,82,46,5,4,1,0,3,1,1,0,3,1,15,11,10,6,5,5,8,11,50,50,2.28,1.61,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.65,3.55,2.0,1.19,1.74,2.72,4.85,1.63,2.23,Wembley Stadium (London)
0,1533477600,Aug 05 2018 - 2:00pm,complete,NaN,Chelsea,Manchester City,Jonathan Moss,NaN,0.0,0.0,0.00,3.00,0,2,2,1,0,1,NaN,"13,58",4,4,0,0,0,0,0,0,0,0,8,19,4,10,4,9,6,11,48,52,0.99,2.14,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.55,3.60,2.1,1.18,1.61,2.50,4.50,1.56,2.30,Wembley Stadium (London)
0,1470422700,Aug 05 2016 - 6:45pm,complete,NaN,Hinckley AFC,Heanor Town FC,NaN,NaN,0.0,0.0,0.50,1.75,0,0,0,0,0,0,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,St. John's Park (Heather- Leicestershire)
1,1470422700,Aug 05 2016 - 6:45pm,complete,NaN,Ascot United,Milton United,NaN,NaN,0.0,0.0,1.75,0.00,2,0,2,0,0,0,NaN,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,Ascot Racecourse (Ascot- Berkshire)


In [11]:
df_players.head()

,full_name,age,birthday,birthday_GMT,league,season,position,Current Club,minutes_played_overall,minutes_played_home,minutes_played_away,nationality,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,assists_home,assists_away,penalty_goals,penalty_misses,clean_sheets_overall,clean_sheets_home,clean_sheets_away,conceded_overall,conceded_home,conceded_away,yellow_cards_overall,red_cards_overall,goals_involved_per_90_overall,assists_per_90_overall,goals_per_90_overall,goals_per_90_home,goals_per_90_away,min_per_goal_overall,conceded_per_90_overall,min_per_conceded_overall,min_per_match,min_per_card_overall,min_per_assist_overall,cards_per_90_overall,rank_in_league_top_attackers,rank_in_league_top_midfielders,rank_in_league_top_defenders,rank_in_club_top_scorer
34,Aapo Halme,22,895795200,1998/05/22,FA Cup,2019/2020,Defender,Barnsley,69,0,69,Finland,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0.00,0.0,0.00,0.0,0.0,0,3.91,23,69,0,0,0.0,-1,-1,-1,13
35,Aaron Anthony Connolly,21,949017600,2000/01/28,FA Cup,2019/2020,Forward,Brighton & Hove Albion,44,44,0,Republic of Ireland,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0.00,0.0,0.00,0.0,0.0,0,2.05,44,44,0,0,0.0,-1,-1,-1,13
36,Aaron Chapman,30,643939200,1990/05/29,FA Cup,2019/2020,Goalkeeper,Tranmere Rovers,90,0,90,England,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0.00,0.0,0.00,0.0,0.0,0,3.00,30,90,0,0,0.0,-1,-1,-1,10
37,Aaron Collins,23,864691200,1997/05/27,FA Cup,2019/2020,Forward,Forest Green Rovers,270,180,90,Wales,3,2,1,1,1,0,0,0,0,0,0,1,1,0,3,2,1,0,0,0.33,0.0,0.33,0.5,0.0,270,1.00,90,90,0,0,0.0,-1,-1,-1,4
38,Aaron Cresswell,31,629683200,1989/12/15,FA Cup,2019/2020,Defender,West Ham United,90,90,0,England,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0.00,0.0,0.00,0.0,0.0,0,1.00,90,90,0,0,0.0,-1,-1,-1,6


## Players matching with Euro data and Footystats data

In [12]:
df_players_euro['full_name'] = df_players_euro['pFName'].values

In [13]:
df_players_euro

,id,pDName,pFName,latinName,tName,tId,teamPlayed,cCode,skill,value,isActive,selPer,mdId,totPts,gS,assist,cS,gC,yC,rC,oG,pS,pC,pE,saves,pM,bR,gOB,mOM,mOMPts,pStatus,matchAtd,trained,isPlayed,selInPer,selOutPer,upcomingMatchesList,currentMatchesList,avgPlayerPts,avgPlayerValue,lastGdPoints,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,full_name
0,63706,C. Ronaldo,Cristiano Ronaldo,Cristiano Ronaldo,Portugal,110,0,POR,4,12.0,1,26.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '1', 'tId': '110', 'tSCode': 'Portug...","[{'mdId': '1', 'tId': '110', 'tSCode': 'Portug...",0.0,0.0,0.0,1.0,0.0,15.0,1.3,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Cristiano Ronaldo
1,250076574,K. Mbappé,Kylian Mbappé,Kylian Mbappe,France,43,0,FRA,4,12.0,1,34.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '1', 'tId': '43', 'tSCode': 'France'...","[{'mdId': '1', 'tId': '43', 'tSCode': 'France'...",0.0,0.0,0.0,1.0,0.0,10.0,0.9,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Kylian Mbappé
2,250016833,H. Kane,Harry Kane,Harry Kane,England,39,0,ENG,4,11.5,1,43.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '1', 'tId': '39', 'tSCode': 'England...","[{'mdId': '1', 'tId': '39', 'tSCode': 'England...",0.0,0.0,0.0,1.0,0.0,17.0,1.7,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Harry Kane
3,250002096,R. Lewandowski,Robert Lewandowski,Robert Lewandowski,Poland,109,0,POL,4,11.5,1,16.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,1.0,1.0,"[{'mdId': '1', 'tId': '109', 'tSCode': 'Poland...","[{'mdId': '1', 'tId': '109', 'tSCode': 'Poland...",0.0,0.0,0.0,1.0,0.0,6.0,0.8,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Robert Lewandowski
4,250010802,R. Lukaku,Romelu Lukaku,Romelu Lukaku,Belgium,13,0,BEL,4,11.0,1,42.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,2.0,1.0,"[{'mdId': '1', 'tId': '13', 'tSCode': 'Belgium...","[{'mdId': '1', 'tId': '13', 'tSCode': 'Belgium...",0.0,0.0,0.0,1.0,0.0,16.0,2.2,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Romelu Lukaku
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,250040465,D. Dibusz,Dénes Dibusz,Denes Dibusz,Hungary,57,0,HUN,1,4.0,1,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,0.0,0.0,"[{'mdId': '1', 'tId': '57', 'tSCode': 'Hungary...","[{'mdId': '1', 'tId': '57', 'tSCode': 'Hungary...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Dénes Dibusz
617,250116727,I. Diveev,Igor Diveev,Igor Diveev,Russia,57451,0,RUS,2,4.0,1,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,0.0,0.0,"[{'mdId': '1', 'tId': '57451', 'tSCode': 'Russ...","[{'mdId': '1', 'tId': '57451', 'tSCode': 'Russ...",0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Igor Diveev
618,250154966,Y. Dyupin,Yuri Dyupin,Yuri Dyupin,Russia,57451,0,RUS,1,4.0,1,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,0.0,0.0,"[{'mdId': '1', 'tId': '57451', 'tSCode': 'Russ...","[{'mdId': '1', 'tId': '57451', 'tSCode': 'Russ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yuri Dyupin
619,250031589,G. Zajkov,Gjoko Zajkov,Gjoko Zajkov,North Macedonia,59205,0,MKD,2,4.0,1,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,,In contention to start next game,0,0.0,0.0,"[{'mdId': '1', 'tId': '59205', 'tSCode': 'Nort...","[{'mdId': '1', 'tId': '59205', 'tSCode': 'Nort...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Gjoko Zajkov


In [14]:
players_exist = df_players_euro[df_players_euro['full_name'].isin(df_players['full_name'].unique())]['full_name'].values

In [15]:
df_players = df_players[df_players['full_name'].isin(players_exist)]
df_players_euro = df_players_euro[df_players_euro['full_name'].isin(players_exist)]

In [16]:
df_players_euro['tName'].value_counts()

Croatia            26
Germany            26
Finland            25
Austria            25
France             25
Switzerland        25
Italy              25
Belgium            25
Slovakia           25
Scotland           24
England            24
Czech Republic     24
Wales              23
Poland             23
Sweden             23
Netherlands        23
Denmark            22
Hungary            21
Ukraine            21
North Macedonia    19
Russia             17
Turkey             17
Spain              13
Portugal           11
Name: tName, dtype: int64

In [17]:
df_players

,full_name,age,birthday,birthday_GMT,league,season,position,Current Club,minutes_played_overall,minutes_played_home,minutes_played_away,nationality,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,assists_home,assists_away,penalty_goals,penalty_misses,clean_sheets_overall,clean_sheets_home,clean_sheets_away,conceded_overall,conceded_home,conceded_away,yellow_cards_overall,red_cards_overall,goals_involved_per_90_overall,assists_per_90_overall,goals_per_90_overall,goals_per_90_home,goals_per_90_away,min_per_goal_overall,conceded_per_90_overall,min_per_conceded_overall,min_per_match,min_per_card_overall,min_per_assist_overall,cards_per_90_overall,rank_in_league_top_attackers,rank_in_league_top_midfielders,rank_in_league_top_defenders,rank_in_club_top_scorer
72,Adam Davies,28,711331200,1992/07/17,FA Cup,2019/2020,Goalkeeper,Stoke City,90,0,90,Wales,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.00,0.00,0.00,0.00,0.00,0,1.00,90,90,0,0,0.00,-1,-1,-1,1
80,Ádám Nagy,25,803347200,1995/06/17,FA Cup,2019/2020,Midfielder,Bristol City,180,90,90,Hungary,2,1,1,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0.00,0.00,0.00,0.00,0.00,0,1.00,90,90,0,0,0.00,-1,-1,-1,13
200,Andreas Christensen,25,829094400,1996/04/10,FA Cup,2019/2020,Defender,Chelsea,145,90,55,Denmark,2,1,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0.00,0.00,0.00,0.00,0.00,0,0.62,145,73,0,0,0.00,-1,-1,-1,24
230,Anssi Jaakkola,34,542592000,1987/03/13,FA Cup,2019/2020,Goalkeeper,Bristol Rovers,270,180,90,Finland,3,2,1,0,0,0,0,0,0,0,0,1,0,1,2,2,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0.67,135,90,0,0,0.00,-1,-1,-1,6
248,Antonio Rüdiger,28,731116800,1993/03/03,FA Cup,2019/2020,Defender,Chelsea,348,90,258,Germany,4,1,3,0,0,0,0,0,0,0,0,2,1,1,3,0,3,1,0,0.00,0.00,0.00,0.00,0.00,0,0.78,116,87,348,0,0.26,109,116,43,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,Thibaut Courtois,29,705542400,1992/05/11,La Liga,2020/2021,Goalkeeper,Real Madrid,3420,1710,1710,Belgium,38,19,19,0,0,0,0,0,0,0,0,17,9,8,28,13,15,0,0,0.00,0.00,0.00,0.00,0.00,0,0.74,122,90,0,0,0.00,364,414,19,21
652,Thomas Lemar,25,816134400,1995/11/12,La Liga,2020/2021,Midfielder,Atlético Madrid,1375,683,692,France,27,13,14,1,1,0,3,2,1,0,0,14,7,7,10,4,6,4,0,0.26,0.20,0.07,0.13,0.00,1375,0.65,138,51,344,458,0.26,196,60,-1,13
656,Tomáš Vaclík,32,607132800,1989/03/29,La Liga,2020/2021,Goalkeeper,Sevilla FC,450,270,180,Czech Republic,5,3,2,0,0,0,0,0,0,0,0,2,1,1,5,3,2,0,0,0.00,0.00,0.00,0.00,0.00,0,1.00,90,90,0,0,0.00,342,276,54,20
657,Toni Kroos,31,631411200,1990/01/04,La Liga,2020/2021,Midfielder,Real Madrid,2121,1013,1108,Germany,28,14,14,3,3,0,10,5,5,0,0,9,5,4,21,11,10,6,0,0.55,0.42,0.13,0.27,0.00,707,0.89,101,76,354,212,0.25,139,5,-1,6


In [18]:
df_players.sort_values(['full_name', 'season'])

,full_name,age,birthday,birthday_GMT,league,season,position,Current Club,minutes_played_overall,minutes_played_home,minutes_played_away,nationality,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,assists_home,assists_away,penalty_goals,penalty_misses,clean_sheets_overall,clean_sheets_home,clean_sheets_away,conceded_overall,conceded_home,conceded_away,yellow_cards_overall,red_cards_overall,goals_involved_per_90_overall,assists_per_90_overall,goals_per_90_overall,goals_per_90_home,goals_per_90_away,min_per_goal_overall,conceded_per_90_overall,min_per_conceded_overall,min_per_match,min_per_card_overall,min_per_assist_overall,cards_per_90_overall,rank_in_league_top_attackers,rank_in_league_top_midfielders,rank_in_league_top_defenders,rank_in_club_top_scorer
4,Aaron Ramsey,30,662169600,1990/12/26,Premier League,2017/2018,Midfielder,Arsenal,1848,976,872,Wales,24,13,11,7,6,1,8,5,3,0,0,10,7,3,22,9,13,0,0,0.73,0.39,0.34,0.55,0.10,264,1.07,84,77,0,231,0.00,44,8,-1,3
6,Aaron Ramsey,30,662169600,1990/12/26,UEFA Europa League,2017/2018,Midfielder,Arsenal,540,270,270,Wales,6,3,3,4,2,2,1,1,0,0,0,1,0,1,6,3,3,1,0,0.83,0.17,0.67,0.67,0.67,135,1.00,90,90,540,540,0.17,36,178,-1,1
3,Aaron Ramsey,30,662169600,1990/12/26,Premier League,2018/2019,Midfielder,Arsenal,1327,689,638,Wales,28,14,14,4,2,2,6,5,1,0,0,7,6,1,12,2,10,0,0,0.68,0.41,0.27,0.26,0.28,332,0.81,111,47,0,221,0.00,69,8,-1,5
8,Aaron Ramsey,30,662169600,1990/12/26,UEFA Europa League,2018/2019,Midfielder,Arsenal,479,267,212,Wales,7,3,4,2,1,1,1,1,0,1,0,6,3,3,1,0,1,0,0,0.56,0.19,0.38,0.34,0.42,240,0.19,479,68,0,479,0.00,165,148,-1,7
1,Aaron Ramsey,30,662169600,1990/12/26,UEFA Nations League,2018/2019,Midfielder,Wales,270,180,90,Wales,3,2,1,1,1,0,0,0,0,0,0,0,0,0,5,3,2,0,0,0.33,0.00,0.33,0.50,0.00,270,1.67,54,90,0,0,0.00,-1,-1,-1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612,Šime Vrsaljko,29,695001600,1992/01/10,UEFA Champions League,2020/2021,Defender,Atlético Madrid,0,0,0,Croatia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0.00,0,0,0,0,0.00,-1,-1,-1,-1
1657,Šime Vrsaljko,29,695001600,1992/01/10,UEFA Nations League,2020/2021,Defender,Croatia,0,0,0,Croatia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0.00,0,0,0,0,0.00,-1,-1,-1,-1
2423,Šime Vrsaljko,29,695001600,1992/01/10,Copa del Rey,2020/2021,Defender,Atlético Madrid,14,0,14,Croatia,1,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,6.43,0.00,6.43,0.00,6.43,14,0.00,0,14,0,0,0.00,-1,-1,-1,3
643,Šime Vrsaljko,29,695001600,1992/01/10,La Liga,2020/2021,Defender,Atlético Madrid,519,103,416,Croatia,9,2,7,0,0,0,0,0,0,0,0,2,1,1,6,1,5,2,0,0.00,0.00,0.00,0.00,0.00,0,1.04,87,58,260,0,0.35,323,268,64,14


In [19]:
df_players[df_players['full_name'].str.contains('Kevin De')].sort_values(['full_name', 'season'])

,full_name,age,birthday,birthday_GMT,league,season,position,Current Club,minutes_played_overall,minutes_played_home,minutes_played_away,nationality,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,assists_home,assists_away,penalty_goals,penalty_misses,clean_sheets_overall,clean_sheets_home,clean_sheets_away,conceded_overall,conceded_home,conceded_away,yellow_cards_overall,red_cards_overall,goals_involved_per_90_overall,assists_per_90_overall,goals_per_90_overall,goals_per_90_home,goals_per_90_away,min_per_goal_overall,conceded_per_90_overall,min_per_conceded_overall,min_per_match,min_per_card_overall,min_per_assist_overall,cards_per_90_overall,rank_in_league_top_attackers,rank_in_league_top_midfielders,rank_in_league_top_defenders,rank_in_club_top_scorer
338,Kevin De Bruyne,29,678063600,1991/06/27,Premier League,2017/2018,Midfielder,Manchester City,3085,1427,1658,Belgium,37,18,19,8,5,3,16,12,4,0,0,18,8,10,20,8,12,2,0,0.70,0.47,0.23,0.32,0.16,386,0.58,154,83,1543,193,0.06,75,4,-1,6
918,Kevin De Bruyne,29,678063600,1991/06/27,UEFA Champions League,2017/2018,Midfielder,Manchester City,667,334,333,Belgium,8,4,4,1,1,0,4,1,3,0,0,4,2,2,8,3,5,4,0,0.67,0.54,0.13,0.27,0.00,667,1.08,83,83,167,167,0.54,207,6,-1,6
374,Kevin De Bruyne,29,678067200,1991/06/28,World Cup,2018,Midfielder,Belgium,540,360,180,Belgium,6,4,2,1,0,1,2,2,0,0,0,2,2,0,5,3,2,1,0,0.50,0.33,0.17,0.00,0.50,540,0.83,108,90,540,270,0.17,54,9,-1,6
322,Kevin De Bruyne,29,678067200,1991/06/28,Premier League,2018/2019,Midfielder,Manchester City,974,477,497,Belgium,19,9,10,2,2,0,2,0,2,0,0,12,6,6,3,1,2,2,0,0.37,0.18,0.18,0.38,0.00,487,0.28,325,51,487,487,0.18,109,75,-1,10
900,Kevin De Bruyne,29,678067200,1991/06/28,UEFA Champions League,2018/2019,Midfielder,Manchester City,247,90,157,Belgium,4,1,3,0,0,0,4,3,1,0,0,1,0,1,5,3,2,0,0,1.46,1.46,0.00,0.00,0.00,0,1.82,49,62,0,62,0.00,-1,-1,-1,21
1623,Kevin De Bruyne,29,678067200,1991/06/28,FA Cup,2019/2020,Midfielder,Manchester City,161,0,161,Belgium,2,0,2,1,0,1,0,0,0,1,0,1,0,1,2,0,2,0,0,0.56,0.00,0.56,0.00,0.56,161,1.12,81,81,0,0,0.00,-1,-1,-1,9
349,Kevin De Bruyne,29,678067200,1991/06/28,Premier League,2019/2020,Midfielder,Manchester City,2800,1403,1397,Belgium,35,17,18,13,9,4,20,12,8,2,0,17,10,7,21,9,12,3,0,1.06,0.64,0.42,0.58,0.26,215,0.68,133,80,933,140,0.10,31,1,-1,4
913,Kevin De Bruyne,29,678067200,1991/06/28,UEFA Champions League,2019/2020,Midfielder,Manchester City,574,317,257,Belgium,7,4,3,2,1,1,2,0,2,1,0,1,0,1,8,6,2,1,0,0.63,0.31,0.31,0.28,0.35,287,1.25,72,82,574,287,0.16,100,48,-1,5
1063,Kevin De Bruyne,29,678067200,1991/06/28,UEFA Euro Qualifiers,2020,Midfielder,Belgium,482,226,256,Belgium,6,3,3,4,3,1,6,1,5,0,0,4,2,2,2,1,1,1,0,1.87,1.12,0.75,1.19,0.35,121,0.37,241,80,482,80,0.19,39,2,-1,4
1661,Kevin De Bruyne,29,678067200,1991/06/28,FA Cup,2020/2021,Midfielder,Manchester City,105,46,59,Belgium,3,1,2,1,0,1,1,1,0,0,0,2,1,1,0,0,0,0,0,1.71,0.86,0.86,0.00,1.53,105,0.00,0,35,0,105,0.00,-1,-1,-1,7


In [20]:
def get_players(team):
    players = df_players[df_players['Current Club'] == team]['full_name'].unique()
    df_team_players = df_players[df_players['full_name'].isin(players)]
    return df_team_players

In [21]:
def get_score_country(team):
    df_team_country = get_players(team)

    df_team_country_attacking = df_team_country[df_team_country['position'].isin(['Forward', 'Midfielder'])][['full_name', 'league', 'season',
           'Current Club', 'appearances_overall', 'goals_overall','assists_overall',
           'goals_involved_per_90_overall']]

    df_team_country_attacking = df_team_country_attacking.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                      'goals_overall': 'sum',
                                                                                     'assists_overall': 'sum',
                                                                                     'goals_involved_per_90_overall':'mean'}).sort_values('goals_involved_per_90_overall', ascending=False).reset_index()

    df_team_country_attacking_score = pd.DataFrame(df_team_country_attacking.head().mean()).transpose()
    df_team_country_attacking_score.columns = [i+'_attacking' for i in df_team_country_attacking_score.columns]

    df_team_country_defending = df_team_country[df_team_country['position'].isin(['Defender', 'Midfielder', 'Goalkeeper'])][['full_name', 'league', 'season',
           'Current Club', 'appearances_overall','clean_sheets_overall', 'conceded_overall', 'conceded_per_90_overall']]

    df_team_country_defending = df_team_country_defending.groupby(['full_name']).agg({'appearances_overall': 'sum',
                                                                                      'clean_sheets_overall': 'sum',
                                                                                     'conceded_overall': 'sum',
                                                                                     'conceded_per_90_overall':'mean'}).sort_values('clean_sheets_overall', ascending=False).reset_index()

    df_team_country_defending_score = pd.DataFrame(df_team_country_defending.head().mean()).transpose()
    df_team_country_defending_score.columns = [i+'_defending' for i in df_team_country_defending_score.columns]
    
    df_team_country_penalty = df_team_country[['full_name', 'league', 'season',
           'Current Club', 'penalty_goals']]

    df_team_country_penalty = df_team_country_penalty.groupby(['full_name']).agg({'penalty_goals': 'mean'}).sort_values('penalty_goals', ascending=False).reset_index()

    df_team_country_penalty_score = pd.DataFrame(df_team_country_penalty.head().mean()).transpose()
    df_team_country_penalty_score.columns = [i+'_penalty' for i in df_team_country_penalty_score.columns]

    df_team_score = df_team_country_attacking_score.join(df_team_country_defending_score).join(df_team_country_penalty_score)
    df_team_score['country'] = team
    return df_team_score

In [22]:
df_all_scores = pd.DataFrame()

for country in df_players_euro['tName'].unique():
    df_temp = get_score_country(country)
    if df_temp.isna().sum().sum() == 0:
        df_all_scores = df_all_scores.append(df_temp)

In [23]:
df_all_scores = df_all_scores[['country']+[i for i in df_all_scores.columns if 'country' not in i]].sort_values('conceded_per_90_overall_defending')

df_all_scores['probability_scoring'] = (df_all_scores['goals_overall_attacking'] + df_all_scores['goals_overall_attacking']) / df_all_scores['appearances_overall_attacking'] * df_all_scores['goals_involved_per_90_overall_attacking']
df_all_scores['probability_concede'] = ((df_all_scores['clean_sheets_overall_defending'] / df_all_scores['appearances_overall_defending']) - (df_all_scores['conceded_overall_defending'] / df_all_scores['appearances_overall_defending'])) * df_all_scores['conceded_per_90_overall_defending']
df_all_scores['probability_concede'] = df_all_scores['probability_concede'].abs()

df_all_scores_final = df_all_scores[['country', 'penalty_goals_penalty', 'probability_scoring', 'probability_concede']].reset_index(drop=True)
df_all_scores_final['country'] = df_all_scores_final['country'].str.lower()
df_all_scores_final.sort_values('probability_scoring')

,country,penalty_goals_penalty,probability_scoring,probability_concede
15,slovakia,0.347222,0.116545,0.712025
17,czech republic,0.080320,0.153981,0.966508
21,russia,0.080000,0.166920,0.693834
11,hungary,0.107222,0.168273,0.684203
19,finland,0.136111,0.183231,0.500551
12,sweden,0.497778,0.213343,0.757798
6,wales,0.059524,0.232717,0.355568
14,switzerland,0.100233,0.275653,0.770397
5,denmark,0.178805,0.276383,0.474091
13,austria,0.221368,0.294201,0.636061


## International matches

In [24]:
df_international_matches = pd.read_csv('data/results.csv')

df_international_matches

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
42100,2021-05-25,Indonesia,Afghanistan,2,3,Friendly,Dubai,United Arab Emirates,True
42101,2021-05-25,Oman,Thailand,1,0,Friendly,Dubai,United Arab Emirates,True
42102,2021-05-27,Turkey,Azerbaijan,2,1,Friendly,Alanya,Turkey,False
42103,2021-05-28,Bahrain,Malaysia,2,0,Friendly,Riffa,Bahrain,False


In [25]:
df_matches = df[(df['home_team_name'].isin(df_players_euro['tName'].unique())) | (df['away_team_name'].isin(df_players_euro['tName'].unique()))].reset_index(drop=True)

df_matches['datetime'] = pd.to_datetime(df_matches['timestamp'], unit='s')

df_matches = df_matches.sort_values('datetime')

df_info = df_matches[df_matches['timestamp'] != 1622314800][['datetime', 'home_team_name',
       'away_team_name', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count']]

df_info.rename(columns={'home_team_name': 'home_team',
       'away_team_name': 'away_team', 'home_team_goal_count': 'home_score',
       'away_team_goal_count' : 'away_score', 'total_goal_count': 'total_goals'}, inplace=True)

df_info = df_info[(df_info['datetime'] < '2021-06-06') & (df_info['datetime'] > '2021-05-29')]
df_info['date'] = df_info['datetime'].dt.date

In [26]:
df_info = df_international_matches[['date', 'home_team', 'away_team', 'home_score', 'away_score']].append(df_info[['date', 'home_team', 'away_team', 'home_score', 'away_score',]])
df_info['total_goals'] = df_info['home_score'] + df_info['away_score']
df_info

,date,home_team,away_team,home_score,away_score,total_goals
0,1872-11-30,Scotland,England,0,0,0
1,1873-03-08,England,Scotland,4,2,6
2,1874-03-07,Scotland,England,2,1,3
3,1875-03-06,England,Scotland,2,2,4
4,1876-03-04,Scotland,England,3,0,3
...,...,...,...,...,...,...
119,2021-06-04,Hungary,Cyprus,1,0,1
120,2021-06-04,Italy,Czech Republic,4,0,4
121,2021-06-05,Russia,Bulgaria,1,0,1
122,2021-06-05,Wales,Albania,0,0,0


In [27]:
df_info['home_team'] = df_info['home_team'].str.lower()
df_info['away_team'] = df_info['away_team'].str.lower()

In [28]:
# avg_home_goal_scored = data[(data['home_team'] == 'manchester city')].home_score.mean()

# avg_away_goal_conceded = data[(data['away_team'] == 'chelsea')].home_score.mean()

#  int(stats.mode(np.random.poisson(1/2*(avg_away_goal_conceded+avg_home_goal_scored),10000))[0])

# 1/2* (data[(data['home_team'] == 'manchester city')].home_score.mean() + data[(data['away_team'] == 'chelsea')].home_score.mean())

In [29]:
def get_probabilty_scoring(home_team, away_team):
    df_home = df_all_scores_final[df_all_scores_final['country'] == home_team]
    df_away = df_all_scores_final[df_all_scores_final['country'] == away_team]
    
    if len(df_home) > 0:
        if len(df_away) > 0:
            home_scoring = df_home['probability_scoring'].values[0] + df_away['probability_concede'].values[0]
            away_scoring = df_away['probability_scoring'].values[0] + df_home['probability_concede'].values[0]
        else:
            home_scoring = df_home['probability_scoring'].values[0] + 1
            away_scoring = 0 + df_home['probability_concede'].values[0]
    else:
        if len(df_away) > 0:
            home_scoring = 0 + df_away['probability_concede'].values[0]
            away_scoring = df_away['probability_scoring'].values[0] + 1
        else:
            home_scoring = 0
            away_scoring = 0
        
    
    return home_scoring, away_scoring

In [30]:
def get_probabilty_penalty(home_team, away_team):
    df_home = df_all_scores_final[df_all_scores_final['country'] == home_team]
    df_away = df_all_scores_final[df_all_scores_final['country'] == away_team]
    
    if len(df_home) > 0:
        if len(df_away) > 0:
            home_scoring = df_home['penalty_goals_penalty'].values[0]
            away_scoring = df_away['penalty_goals_penalty'].values[0]
        else:
            home_scoring = df_home['penalty_goals_penalty'].values[0]
            away_scoring = 0
    else:
        if len(df_away) > 0:
            home_scoring = 0
            away_scoring = df_away['penalty_goals_penalty'].values[0]
        else:
            home_scoring = 0
            away_scoring = 0
    
    return home_scoring, away_scoring

In [31]:
def round_school(x):
    i, f = divmod(x, 1)
    return int(i + ((f >= 0.5) if (x > 0) else (f > 0.5)))

In [32]:
def predict_score(home_team, away_team, extra_time_penalty):
    ht = (' '.join(home_team.split())).lower()
    at = (' '.join(away_team.split())).lower()
    
    if len(df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)]) > 20:
        
        avg_home_score = df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)].home_score.mean()
        avg_away_score = df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)].away_score.mean()
        
#         home_goal = int(stats.mode(np.random.poisson(avg_home_score,10000))[0])                    
#         away_goal = int(stats.mode(np.random.poisson(avg_away_score,10000))[0])
        home_goal = np.mean(np.random.poisson(avg_home_score,10000))                
        away_goal = np.mean(np.random.poisson(avg_away_score,10000))
        
    else:
        avg_away_goal_conceded = df_info[(df_info.away_team ==at)].home_score.mean()
        avg_home_goal_scored   = df_info[(df_info.home_team ==ht)].home_score.mean()
#         home_goal = int(stats.mode(np.random.poisson(1/2*(avg_away_goal_conceded+avg_home_goal_scored),10000))[0])
        home_goal = np.mean(np.random.poisson(1/2*(avg_away_goal_conceded+avg_home_goal_scored),10000))
    
        avg_home_goal_conceded = df_info[(df_info.home_team ==ht)].away_score.mean()
        avg_away_goal_scored   = df_info[(df_info.away_team ==at)].away_score.mean()
#         away_goal = int(stats.mode(np.random.poisson(1/2*(avg_home_goal_conceded+avg_away_goal_scored),10000))[0])
        away_goal = np.mean(np.random.poisson(1/2*(avg_home_goal_conceded+avg_away_goal_scored),10000))
    
    prob_home_score, prob_away_score = get_probabilty_scoring(ht, at)
    prob_home_score_penalty, prob_away_score_penalty = get_probabilty_penalty(ht, at)
    
    home_goal = home_goal + prob_home_score + (prob_home_score_penalty * 0.1)
    away_goal = away_goal + prob_away_score + (prob_away_score_penalty * 0.1)
    
    home_goal = round_school(home_goal)
    away_goal = round_school(away_goal)
    
#     home_goal = int(home_goal + prob_home_score + (prob_home_score_penalty * 0.2))
#     away_goal = int(away_goal + prob_away_score + (prob_away_score_penalty * 0.2))
    
#     print(f'They have played {len(df_info[(df_info.home_team ==ht) & (df_info.away_team ==at)])} matches')
    
    if home_goal > away_goal:
        print(f'{home_team.capitalize()} win. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
    elif home_goal < away_goal:
        print(f'{away_team.capitalize()} win. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
    elif home_goal == away_goal:
        if extra_time_penalty:
            if prob_home_score_penalty > prob_away_score_penalty:
                print(f'Penalty won by {home_team.capitalize()}')
            elif prob_home_score_penalty < prob_away_score_penalty:
                print(f'Penalty won by {away_team.capitalize()}')
            else:
                if prob_home_score > prob_away_score:
                    print(f'Penalty won by {home_team.capitalize()}')
                else:
                    print(f'Penalty won by {away_team.capitalize()}')
        else:
            print(f'Draw. The scoreline is {home_team.capitalize()} {home_goal}:{away_goal} {away_team.capitalize()}')
    return home_goal, away_goal

In [33]:
predict_score('portugal', 'belgium', True)

Portugal win. The scoreline is Portugal 3:2 Belgium


(3, 2)

In [34]:
def update_table(df_table, home_team, away_team, home_score, away_score):
    if home_team not in df_table['country'].values:
        df_temp = pd.DataFrame()
        df_temp['country'] = [home_team]
        if home_score > away_score:
            df_temp['win'] = [1]
            df_temp['draw'] = [0]
            df_temp['lose'] = [0]
            df_temp['gf'] = [home_score]
            df_temp['ga'] = [away_score]
            df_temp['pts'] = [3]
        elif home_score < away_score:
            df_temp['win'] = [0]
            df_temp['draw'] = [0]
            df_temp['lose'] = [1]
            df_temp['gf'] = [home_score]
            df_temp['ga'] = [away_score]
            df_temp['pts'] = [0]
        else:
            df_temp['win'] = [0]
            df_temp['draw'] = [1]
            df_temp['lose'] = [0]
            df_temp['gf'] = [home_score]
            df_temp['ga'] = [away_score]
            df_temp['pts'] = [1]
        df_table = df_table.append(df_temp)
    else:
        if home_score > away_score:
            df_temp = df_table[df_table['country'] == home_team]
            win = df_temp['win'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == home_team, 'win'] = win+1
            df_table.loc[df_table['country'] == home_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == home_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == home_team, 'pts'] = pts+3
        elif home_score < away_score:
            df_temp = df_table[df_table['country'] == home_team]
            lose = df_temp['lose'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == home_team, 'lose'] = lose+1
            df_table.loc[df_table['country'] == home_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == home_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == home_team, 'pts'] = pts+0
        else:
            df_temp = df_table[df_table['country'] == home_team]
            draw = df_temp['draw'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == home_team, 'draw'] = draw+1
            df_table.loc[df_table['country'] == home_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == home_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == home_team, 'pts'] = pts+1
            
    if away_team not in df_table['country'].values:
        df_temp = pd.DataFrame()
        df_temp['country'] = [away_team]
        if away_score > home_score:
            df_temp['win'] = [1]
            df_temp['draw'] = [0]
            df_temp['lose'] = [0]
            df_temp['gf'] = [away_score]
            df_temp['ga'] = [home_score]
            df_temp['pts'] = [3]
        elif away_score < home_score:
            df_temp['win'] = [0]
            df_temp['draw'] = [0]
            df_temp['lose'] = [1]
            df_temp['gf'] = [away_score]
            df_temp['ga'] = [home_score]
            df_temp['pts'] = [0]
        else:
            df_temp['win'] = [0]
            df_temp['draw'] = [1]
            df_temp['lose'] = [0]
            df_temp['gf'] = [away_score]
            df_temp['ga'] = [home_score]
            df_temp['pts'] = [1]
        df_table = df_table.append(df_temp)
    else:
        if home_score < away_score:
            df_temp = df_table[df_table['country'] == away_team]
            win = df_temp['win'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == away_team, 'win'] = win+1
            df_table.loc[df_table['country'] == away_team, 'gf'] = gf+away_score
            df_table.loc[df_table['country'] == away_team, 'ga'] = ga+home_score
            df_table.loc[df_table['country'] == away_team, 'pts'] = pts+3
        elif home_score > away_score:
            df_temp = df_table[df_table['country'] == away_team]
            lose = df_temp['lose'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == away_team, 'lose'] = lose+1
            df_table.loc[df_table['country'] == away_team, 'gf'] = gf+away_score
            df_table.loc[df_table['country'] == away_team, 'ga'] = ga+home_score
            df_table.loc[df_table['country'] == away_team, 'pts'] = pts+0
        else:
            df_temp = df_table[df_table['country'] == away_team]
            draw = df_temp['draw'].values[0]
            gf = df_temp['gf'].values[0]
            ga = df_temp['ga'].values[0]
            pts = df_temp['pts'].values[0]
            df_table.loc[df_table['country'] == away_team, 'draw'] = draw+1
            df_table.loc[df_table['country'] == away_team, 'gf'] = gf+home_score
            df_table.loc[df_table['country'] == away_team, 'ga'] = ga+away_score
            df_table.loc[df_table['country'] == away_team, 'pts'] = pts+1
    return df_table

In [35]:
df_matches_euro = df_matches_euro[['htName', 'atName', 'mdName', 'groupName']]
df_matches_euro.rename(columns={'htName': 'home_team', 'atName': 'away_team'}, inplace=True)
df_matches_euro

,home_team,away_team,mdName,groupName
0,Turkey,Italy,Match 1,Group A
1,Wales,Switzerland,Match 4,Group A
2,Denmark,Finland,Match 3,Group B
3,Belgium,Russia,Match 2,Group B
4,England,Croatia,Match 7,Group D
5,Austria,North Macedonia,Match 5,Group C
6,Netherlands,Ukraine,Match 6,Group C
7,Scotland,Czech Republic,Match 8,Group D
8,Poland,Slovakia,Match 9,Group E
9,Spain,Sweden,Match 10,Group E


In [36]:
df_matches_euro['home_team'] = df_matches_euro['home_team'].str.lower()
df_matches_euro['away_team'] = df_matches_euro['away_team'].str.lower()

In [37]:
dict_tables = {}

In [38]:
for index, data in df_matches_euro[:36].iterrows():
    if data['groupName'] in dict_tables:
        df_table = dict_tables[data['groupName']]
        home_score, away_score = predict_score(data['home_team'], data['away_team'], False)
        df_table = update_table(df_table, data['home_team'], data['away_team'], home_score, away_score)
        dict_tables[data['groupName']] = df_table.sort_values('pts', ascending=False).reset_index(drop=True)
    else:
        df_table = pd.DataFrame(columns=['country', 'win', 'draw', 'lose', 'gf', 'ga', 'pts'])
        home_score, away_score = predict_score(data['home_team'], data['away_team'], False)
        df_table = update_table(df_table, data['home_team'], data['away_team'], home_score, away_score)
        dict_tables[data['groupName']] = df_table.sort_values('pts', ascending=False).reset_index(drop=True)

Italy win. The scoreline is Turkey 2:3 Italy
Wales win. The scoreline is Wales 3:2 Switzerland
Denmark win. The scoreline is Denmark 4:1 Finland
Belgium win. The scoreline is Belgium 3:2 Russia
England win. The scoreline is England 3:2 Croatia
Austria win. The scoreline is Austria 3:2 North macedonia
Netherlands win. The scoreline is Netherlands 3:2 Ukraine
Scotland win. The scoreline is Scotland 3:2 Czech republic
Poland win. The scoreline is Poland 3:2 Slovakia
Spain win. The scoreline is Spain 3:2 Sweden
Draw. The scoreline is Hungary 2:2 Portugal
France win. The scoreline is France 3:2 Germany
Draw. The scoreline is Finland 2:2 Russia
Turkey win. The scoreline is Turkey 3:2 Wales
Italy win. The scoreline is Italy 4:2 Switzerland
Ukraine win. The scoreline is Ukraine 3:2 North macedonia
Draw. The scoreline is Denmark 3:3 Belgium
Netherlands win. The scoreline is Netherlands 3:2 Austria
Sweden win. The scoreline is Sweden 3:2 Slovakia
Croatia win. The scoreline is Croatia 3:2 Czech r

In [39]:
from IPython.core import display as ICD

In [40]:
for group in sorted(list(dict_tables.keys())):
    print(group)
    ICD.display(dict_tables[group])

Group A


,country,win,draw,lose,gf,ga,pts
0,italy,3,0,0,10,5,9
1,turkey,1,1,1,8,8,4
2,wales,1,0,2,6,8,3
3,switzerland,0,1,2,7,10,1


Group B


,country,win,draw,lose,gf,ga,pts
0,belgium,2,1,0,9,7,7
1,denmark,1,2,0,9,6,5
2,russia,0,2,1,6,7,2
3,finland,0,1,2,5,9,1


Group C


...

,country,win,draw,lose,gf,ga,pts
0,england,3,0,0,10,6,9
1,croatia,2,0,1,8,7,6
2,scotland,1,0,2,7,9,3
3,czech republic,0,0,3,6,9,0


Group E


,country,win,draw,lose,gf,ga,pts
0,spain,2,1,0,8,6,7
1,poland,1,1,1,8,8,4
2,sweden,1,1,1,8,8,4
3,slovakia,0,1,2,6,8,1


Group F


,country,win,draw,lose,gf,ga,pts
0,portugal,2,1,0,8,6,7
1,france,2,0,1,8,7,6
2,germany,1,0,2,7,8,3
3,hungary,0,1,2,6,8,1


## Round of 16

In [48]:
predict_score('belgium', 'sweden', True)

Belgium win. The scoreline is Belgium 3:2 Sweden


(3, 2)

In [49]:
predict_score('italy', 'ukraine', True)

Italy win. The scoreline is Italy 3:2 Ukraine


(3, 2)

In [50]:
predict_score('portugal', 'wales', True)

Portugal win. The scoreline is Portugal 3:1 Wales


(3, 1)

In [51]:
predict_score('croatia', 'poland', True)

Croatia win. The scoreline is Croatia 3:2 Poland


(3, 2)

In [52]:
predict_score('spain', 'russia', True)

Spain win. The scoreline is Spain 3:2 Russia


(3, 2)

In [54]:
predict_score('england', 'france', True)

Penalty won by France


(3, 3)

In [55]:
predict_score('netherlands', 'germany', True)

Penalty won by Netherlands


(3, 3)

In [56]:
predict_score('turkey', 'denmark', True)

Turkey win. The scoreline is Turkey 3:2 Denmark


(3, 2)

## Quarter Finals

In [66]:
predict_score('belgium', 'italy', True)

Penalty won by Italy


(3, 3)

In [58]:
predict_score('portugal', 'croatia', True)

Portugal win. The scoreline is Portugal 3:2 Croatia


(3, 2)

In [59]:
predict_score('spain', 'france', True)

Spain win. The scoreline is Spain 3:2 France


(3, 2)

In [60]:
predict_score('netherlands', 'turkey', True)

Netherlands win. The scoreline is Netherlands 3:2 Turkey


(3, 2)

## Semi Finals

In [61]:
predict_score('italy', 'portugal', True)

Italy win. The scoreline is Italy 3:2 Portugal


(3, 2)

In [62]:
predict_score('spain', 'netherlands', True)

Spain win. The scoreline is Spain 3:2 Netherlands


(3, 2)

## Finals

In [64]:
predict_score('spain', 'italy', True)

Penalty won by Italy


(2, 2)

<img src="bracket_landscape.png">